# [SimSiam](https://arxiv.org/abs/2011.10566) on CIFAR-10

In [1]:
from os import makedirs

import torch
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision.models import resnet18

from models import SimSiam
from trainer import SSL_Trainer
from utils import SSL_CIFAR10

# Define hyperparameters
data_root = data_root = some root so data folder
save_root = './results/simsiam/'

dl_kwargs = {'batch_size': 512, 'shuffle': True, 'num_workers': 2}

# Define data
ssl_data = SSL_CIFAR10(data_root,'SimSiam', dl_kwargs)

# general training params
train_params = {'save_root': save_root, 'num_epochs': 800, 'optimizer': SGD,
                'scheduler': CosineAnnealingLR, 'warmup_epochs': 10, 'iter_scheduler':True,
                'evaluate_at': [100,200,400,600], 'verbose':True}

# params of optimizer
# From original paper
optim_params = {'lr':0.03 * dl_kwargs['batch_size']/256,
                'weight_decay': 5e-4,
                'momentum' : 0.9}

# params of scheduler
scheduler_params = {'T_max': (train_params['num_epochs']-train_params['warmup_epochs'])*len(ssl_data.train_dl)}
                    # 'eta_min':1e-4} in orginal implementation

# Set parameters for fitting linear protocoler
eval_params  = {'lr':1e-2, 'num_epochs': 25, 'milestones': [12,20]}

# Get device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Print Device Type
if torch.cuda.is_available():
    print(f"Program running on {torch.cuda.get_device_name(device)}")
else:
    print("Program running on CPU")
    
# Create folder if it does not exists
makedirs(save_root, exist_ok=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Program running on NVIDIA A100-PCIE-40GB


# SimSiam

In [ ]:
# Define Model
resnet = resnet18(zero_init_residual=True)

simsiam = SimSiam(resnet,
                  projector_hidden = (2048, 2048),
                  predictor_hidden = (512, 2048))

# Define Trainer
cifar10_trainer = SSL_Trainer(simsiam, ssl_data, device)

# Train
cifar10_trainer.train(**train_params, optim_params=optim_params,
                      scheduler_params=scheduler_params, eval_params=eval_params)

/home/fcfschulz/miniconda3/envs/pg_high/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0, Loss: 0.9814488555967193, Time epoch: 187.97060108184814
Epoch: 1, Loss: 0.8999253001409707, Time epoch: 203.1080060005188
Epoch: 2, Loss: 0.7540117293289027, Time epoch: 200.95217490196228
Epoch: 3, Loss: 0.6343861074791741, Time epoch: 202.13703632354736
Epoch: 4, Loss: 0.5814269277238354, Time epoch: 203.0395324230194
Epoch: 5, Loss: 0.5513902030040309, Time epoch: 200.9306755065918
Epoch: 6, Loss: 0.5281851998309499, Time epoch: 199.75597286224365
Epoch: 7, Loss: 0.5052382638773967, Time epoch: 198.20007419586182
Epoch: 8, Loss: 0.47937036851017745, Time epoch: 205.35928893089294
Epoch: 9, Loss: 0.4586545290406217, Time epoch: 207.7198724746704
Epoch: 10, Loss: 0.443931519985199, Time epoch: 208.11708688735962
